In [ ]:
pip install scikit learn

In [ ]:
import sklearn

In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

def get_color(idx):
    np.random.seed(idx)
    return tuple(int(x) for x in np.random.randint(0, 255, 3))

# Load MiDaS model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")
midas.to(device).eval()
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms").small_transform

# Load YOLO model
model = YOLO('best_mosaic.pt')

# Load image
frame = cv2.imread(r"traffic_wala_dataset\valid\images\test_mp4-13_jpg.rf.98cd77f75c4492f8f103aaf4ce2ca8f8.jpg")
if frame is None:
    raise FileNotFoundError("Image not found.")

# Depth estimation
input_batch = midas_transforms(frame).to(device)
with torch.no_grad():
    prediction = midas(input_batch)
    depth_map = prediction.squeeze().cpu().numpy()

# Detection
results = model(frame)

# Step 1: Extract vehicle centers
vehicle_centers = []
bbox_data = []  # Store extra info for drawing
for box in results[0].boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
    vehicle_centers.append([cx, cy])
    bbox_data.append((x1, y1, x2, y2, cx, cy))

vehicle_centers = np.array(vehicle_centers)

# Step 2: DBSCAN clustering
if len(vehicle_centers) > 0:
    clustering = DBSCAN(eps=100, min_samples=2).fit(vehicle_centers)
    labels = clustering.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    colors = plt.cm.get_cmap('tab20', n_clusters if n_clusters > 0 else 1)

    # Step 3: Draw cluster circles
    for (cx, cy), label in zip(vehicle_centers, labels):
        if label == -1:
            color = (128, 128, 128)  # noise
        else:
            c = colors(label)
            color = (int(c[2]*255), int(c[1]*255), int(c[0]*255))
        cv2.circle(frame, (cx, cy), 10, color, -1)
        cv2.putText(frame, f'{label}', (cx - 10, cy - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Step 4: Compute depth distances and proximity warnings
vehicle_depths = {}
too_close_pairs = set()
for idx, (x1, y1, x2, y2, cx, cy) in enumerate(bbox_data):
    cx = np.clip(cx, 0, depth_map.shape[1] - 1)
    cy = np.clip(cy, 0, depth_map.shape[0] - 1)
    depth_value = depth_map[cy, cx]
    vehicle_depths[idx] = (cx, cy, depth_value)

# Compare pairwise distances
for i in range(len(vehicle_depths)):
    for j in range(i + 1, len(vehicle_depths)):
        (x1, y1, d1) = vehicle_depths[i]
        (x2, y2, d2) = vehicle_depths[j]
        pixel_dist = np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
        depth_diff = abs(d1 - d2)
        combined_dist = pixel_dist + (depth_diff * 1000)
        if combined_dist < 300:
            too_close_pairs.add(i)
            too_close_pairs.add(j)

# Step 5: Draw final annotations
for idx, (x1, y1, x2, y2, cx, cy) in enumerate(bbox_data):
    color = get_color(idx)
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
    cv2.putText(frame, f"ID: {idx}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    if idx in too_close_pairs:
        cv2.putText(frame, "Too Close!", (cx, cy - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

# Vehicle count
cv2.putText(frame, f"Total Vehicles: {len(bbox_data)}", (20, 100),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

# Display
cv2.imshow("Traffic Clustered", frame)
cv2.imwrite("output_image_result.jpg", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()
